### API

De modellen van HuggingFace kunnen via een *inference api* worden aangesproken. Op deze manier wordt een getraind taalmodel aangesproken zonder dat deze lokaal moet worden opgeslaan. Bij een prototype bespaart dit zowel tijd als memory space.

In [2]:
import configparser
import time, json, requests, csv
from langdetect import detect
from googletrans import Translator
import pandas as pd

huggingfacemodels = {
    'sc':"https://api-inference.huggingface.co/models/haining/scientific_abstract_simplification",
    'bart':"https://api-inference.huggingface.co/models/facebook/bart-large-cnn",
    'bart-sc': "https://api-inference.huggingface.co/models/sambydlo/bart-large-scientific-lay-summarisation",
    't5':"https://api-inference.huggingface.co/models/mrm8488/t5-base-finetuned-summarize-news",
    'kis': "https://api-inference.huggingface.co/models/philippelaban/keep_it_simple",
    'gpt-2':'https://api-inference.huggingface.co/models/gpt2',
    'gpt-2-ft': "https://api-inference.huggingface.co/models/gavin124/gpt2-finetuned-cnn-summarization-v2",
    'peg':'https://api-inference.huggingface.co/models/google/pegasus-xsum',
    'peg-par':'https://api-inference.huggingface.co/models/tuner007/pegasus_paraphrase'
}


import requests

In [3]:
config = configparser.ConfigParser()
config.read('../web-app/config.ini')
global huggingface_api_key
huggingface_api_key = config['huggingface']['api_key']

In [4]:
global google_translate
google_translate = Translator()

In [5]:
def query(payload, API_URL):
    headers = {"Authorization": f"Bearer {huggingface_api_key}"}
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [6]:
from bs4 import BeautifulSoup
import numpy as np
import spacy 

languages = {
    'nl':'nl_core_news_sm',
    'en':'en_core_web_md'
}

def query(payload, API_URL):
    headers = {"Authorization": f"Bearer {huggingface_api_key}"}
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
    

def summarize(text, lm_key):
        ratio = 0.5
        
        soup = BeautifulSoup(text, 'html.parser')
        gt = Translator()
        
        h3_tags = soup.find_all('h3')

        split_text = {}
        for i, tag in enumerate(h3_tags):
            key = tag.text
            value = ""
            for sibling in tag.next_siblings:
                if sibling.name == 'h3':
                    break
                value += str(sibling.get_text()).replace('\n',' ')
            split_text[key] = value


        result_dict = {}
        for key in split_text.keys():
            text = split_text[key]

            origin_lang = detect(text)
            nlp = spacy.load(languages.get(origin_lang, 'en'))
            doc = nlp(text)

            sentences = []
            for s in doc.sents:
                try:
                    if origin_lang in ['fr','nl','de']:
                        text = gt.translate(text=str(s),src=origin_lang,dest='en').text
                    elif origin_lang not in ['fr','nl','de','en']:
                        text = gt.translate(text=s,src='en',dest='en').text
                    else:
                        text = s
                    
                    sentences.append(text)
                except Exception as e:
                    continue

            API_URL = huggingfacemodels.get(lm_key)

            sentences = np.array(sentences)
            pad_size = 3 - (sentences.size % 3)
            padded_a = np.pad(sentences, (0, pad_size), mode='empty')
            paragraphs = padded_a.reshape(-1, 3)


            output = []
            text = ""
            for i in paragraphs:
                length = len(str(i))
                result = query({"inputs": str(i),"parameters": {"repetition_penalty": 4.0,"max_length": length*ratio,"min_length":1},"options":{"wait_for_model":True}}, API_URL)

                try:
                    if 'generated_text' in result[0]:
                        text = result[0].get('generated_text')

                    if 'summary_text' in result[0]:
                        text = result[0].get('summary_text')
                except:
                    print('server outage')

                try:
                    if origin_lang != 'en':
                        text = gt.translate(text=str(text),src="en", dest=origin_lang).text 
                except TypeError as e:
                    text = ''

                output.append(text)

            result_dict[key] = output

        return(result_dict)

In [7]:
sentences = [
    "De gedesoriënteerde amfibie verorbert haar nutritievolle maaltijd. Desalniettemin werd de implementatie van de voorgestelde maatregelen vertraagd door bureaucratische obstakels. Ondanks het feit dat de resultaten van het onderzoek veelbelovend waren, waren er nog steeds enkele beperkingen die verder onderzoek vereisten.",
    "De commissie concludeerde dat de huidige regelgeving onvoldoende was om de problemen adequaat aan te pakken. Het is van cruciaal belang dat er rekening wordt gehouden met de potentiële gevolgen van dergelijke beslissingen op lange termijn. De complexiteit van de situatie vereiste een zorgvuldige afweging van alle beschikbare opties. De voortdurende ontwikkelingen in de technologie hebben geleid tot aanzienlijke veranderingen in de manier waarop we communiceren en informatie delen.",
    "Het is belangrijk om te benadrukken dat dergelijke initiatieven alleen succesvol kunnen zijn als er voldoende steun is van alle betrokken partijen. De analyse toonde aan dat er aanzienlijke verschillen waren tussen de verschillende groepen in termen van hun percepties en attitudes. Er is een groeiende bezorgdheid over de impact van klimaatverandering op onze planeet en de noodzaak om actie te ondernemen. Het is essentieel dat we ons blijven inzetten voor het bevorderen van duurzame ontwikkeling en het beschermen van onze natuurlijke hulpbronnen."
]

for key in huggingfacemodels.keys():
    API_URL = huggingfacemodels.get(key)
    for s in sentences:
        result = query({"inputs": str(s),"parameters": {"repetition_penalty": 4.0,"min_length":1},"options":{"wait_for_model":True}}, API_URL)
        print(f'{key} --> {result}')

sc --> {'error': 'Model haining/scientific_abstract_simplification time out'}
sc --> {'error': 'Model haining/scientific_abstract_simplification time out'}
sc --> {'error': 'Model haining/scientific_abstract_simplification time out'}
bart --> [{'summary_text': 'De gedesoriënteerde amfibie verorbert haar nutritievolle maaltijd. Desalniettemin werd de implementatie van de voorgestelde maatregelen vertraagd door bureaucratische obstakels. Ondanks het feit dat de resultaten van hebben veelbelovend waren, waren er nog steeds enkele beperkingen die verder onderzoek vereisten.'}]
bart --> [{'summary_text': 'De commissie concludeerde dat de huidige regelgeving onvoldoende was om de problemen adequaat aan te pakken. Het is van cruciaal belang dat er rekening wordt gehouden met de potentiële gevolgen van dergelijke beslissingen op lange termijn. De voortdurende ontwikkelingen in de technologie hebben geleid tot aanzienlijke veranderingen in the manier waarop we communiceren, delen.'}]
bart --> [

In [21]:
df = pd.read_csv('output.csv', encoding = "ISO-8859-1", header=None, names=['model','sentence'])
df.head()

,model,sentence
0,sc,De commissie concludeerde dat de huidige regel...
1,sc,De gedesoriënteerde amfibie verorbert haar nut...
2,sc,Desalniettemin werd de implementatie van de vo...
3,sc,Ondanks het feit dat de resultaten van het ond...
4,sc,De commissie concludeerde dat de huidige regel...


Keuze tussen taalmodellen:

SC en BART-SC scoren subjectief hoger dan de andere modellen. Zij bekijken enkel de gekregen zin. De andere modellen zijn eerder geneigd om extra tekst toe te voegen. Er kan niet achterhaald worden waarom dat deze extra tekst wordt meegegeven.
BART-SC kan bijzaak behouden, terwijl SC sneller de neiging heeft om enkel de kernzaak te behouden in de vereenvoudigde tekst.
Bij de API's moet er expliciet worden aangegeven om welke  transformatie dit gaat. 

Ontwikkelaars kunnen voor algemene samenvattings- en vereenvoudigingstaken gebruik maken van algemene taalmodellen die vrij beschikbaar op HuggingFace of dergelijke platforms terug te vinden zijn. GPT-3 blinkt uit in gepersonaliseerde vereenvoudigings- en samenvattingstaken. Engelstalige prompts die expliciet de uitvoertaal vermelden zijn nauwkeuriger dan Nederlandstalige prompts. 